In [ ]:
def up_or_down(centroid, line, line_direction, threshold):
    '''
    centroid: the center of object
    kind_of_line: we choose our direction
    line: the thresh for centroid
    threshold: a maximum 
    '''
    if line_direction == 'horizon':
        point = centroid[0]
        if round(point-line) <= 5:
            return 'up'
    elif line_direction == 'vectorize':
        point = centroid[1]
        if round(point-line) <= thresh
    else:
        raise "Please choose your direction is 'horizon' or 'vectorize'"

In [3]:
import cv2
import numpy as np


def get_output_layers(net):
    '''
    get all output layer names: with yolov3 is yolo_82, 94 and 106
    ''' 
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidences, x, y, x_plus_w, y_plus_h):
    '''
    draw a bounding box around object
    '''
    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def bounding_pic(net, image, scale, size):
    '''
    net: the model reading by open-cv (maybe YoLo or SSD)
    image: image need to bouding box
    scale: image pixel multiply with this
    size: a tuple contain size of image
    '''
    Width = image.shape[1]
    Height = image.shape[0]
    # Resize picture to 416x416, because YOLO take in 416x416
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input is resized picture
    net.setInput(blob)
    # last layer of Yolo model
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.6
    # maybe our model will detect many bouding box for an object, this threshold help us take the box with equal 
    #            or higher propability
    nms_threshold = 0.4

    '''
    out is a 2D tensor like (number_of_objects, score_of_each_classes), with first five element in each row is special, 
    take e.g: out[0] = temp:
        + temp[0]: x_center of that object
        + temp[1]: y_center of that object
        + temp[2]: width of that object
        + temp[3]: height of that object
        + temp[4]: unknow value
        + from 5 to above is the score of that object to each classes => COCO have 80 class so each row contain 85 element,
            will be 15 with CIFAR,and 1005 with IMAGENET  
    '''
    for out in outs:
        for detection in out:
            scores = detection[5:]
            # get the highest score to determine its label
            class_id = np.argmax(scores)
            if class_id not in [0,1,2,3,7]:
                continue
            else:
                # score of that object, make sure more than 50% correct label
                confidence = scores[class_id]
                if confidence > 0.5:
                    # scale again with w and h
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    # remember it return x_center and y_center, not x,y, so we need to find x,y
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

    # detect bouding box around objects
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    # set the counting line
    lineThickness = 2
    x1,y1,x2,y2 = (322,580,1030,580)
    image = cv2.line(image, (x1, y1), (x2, y2), (0,255,0), lineThickness)
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    return image

# image_path = 'motorbike.jpg'
config = 'YOLOv3-608\\yolov3.cfg'
name = 'coco.names'
weight = 'YOLOv3-608\\yolov3.weights'

# image = cv2.imread(image_path)

classes = None

with open(name, 'r') as f:
    # generate all classes of COCO, bicycle ind = 1, car ind = 2 and motorbike ind = 3
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Read the model
net = cv2.dnn.readNet(weight, config)

# take shape of image in order to scale it to 416x416, first layer of Yolo CNN

scale = 0.00392



cap = cv2.VideoCapture('C:\\Users\\ADMINS\\Desktop\\front.avi')

count = 0
while(True):
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    thresh = 15
    if count%thresh==0:
        # Our operations on the frame come here
        img = bounding_pic(net, frame, scale, (416,416))

        # Display the resulting frame
        cv2.imshow('frame',img)
    count += 1
    if count == thresh*30:
        count = 0
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [5]:
import torch

ModuleNotFoundError: No module named 'torch'

In [1]:
import cv2
import numpy as np


def get_output_layers(net):
    '''
    get all output layer names: with yolov3 is yolo_82, 94 and 106
    ''' 
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidences, x, y, x_plus_w, y_plus_h):
    '''
    draw a bounding box around object
    '''
    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def find_centroid(net, image, scale, size):
    '''
    Input:
        + net: the model reading by open-cv (maybe YoLo or SSD)
        + image: image need to bouding box
        + scale: image pixel multiply with this
        + size: a tuple contain size of image
    Output:
        + A list of centroid of all objects in image
    '''
    Width = image.shape[1]
    Height = image.shape[0]
    # Resize picture to 416x416, because YOLO take in 416x416
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input is resized picture
    net.setInput(blob)
    # last layer of Yolo model
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.6
    # maybe our model will detect many bouding box for an object, this threshold help us take the box with equal 
    #            or higher propability
    nms_threshold = 0.4

    '''
    out is a 2D tensor like (number_of_objects, score_of_each_classes), with first five element in each row is special, 
    take e.g: out[0] = temp:
        + temp[0]: x_center of that object
        + temp[1]: y_center of that object
        + temp[2]: width of that object
        + temp[3]: height of that object
        + temp[4]: unknow value
        + from 5 to above is the score of that object to each classes => COCO have 80 class so each row contain 85 element,
            will be 15 with CIFAR,and 1005 with IMAGENET  
    '''
    centroid = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            # get the highest score to determine its label
            class_id = np.argmax(scores)
            if class_id not in [0,1,2,3,7]:
                continue
            else:
                # score of that object, make sure more than 50% correct label
                confidence = scores[class_id]
                if confidence > 0.5:
                    # scale again with w and h
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    centroid.append(np.array([center_x,center_y]))
    lineThickness = 2
    x1,y1,x2,y2 = (322,580,1030,580)
    image = cv2.line(image, (x1, y1), (x2, y2), (0,255,0), lineThickness)
    return centroid, image

config = 'YOLOv3-320\\yolov3.cfg'
name = 'coco.names'
weight = 'YOLOv3-320\\yolov3.weights'

# image = cv2.imread(image_path)

classes = None

with open(name, 'r') as f:
    # generate all classes of COCO, bicycle ind = 1, car ind = 2 and motorbike ind = 3
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Read the model
net = cv2.dnn.readNet(weight, config)

# take shape of image in order to scale it to 416x416, first layer of Yolo CNN

scale = 0.00392



cap = cv2.VideoCapture('C:\\Users\\Personal Computer\\Desktop\\front.avi')

count = 0
ID_count = 0
c_dict = {}
while(True):
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Take picture for every thresh frames
    thresh_fps = 15
    # Take the smaller distance from this thresh to decide the ID
    thresh_dist = 100
    # First frame, only take position and give ID to each vehicle
    if count == 0:
        all_centroid, _ = find_centroid(net, frame, scale, (416,416))
        for centroid in all_centroid:
            c_dict[ID_count] = centroid
            ID_count += 1
        count += 1
    # from second frame to above, take the previous data for tracking
    if count%thresh_fps==0:
        # find all centroid in each frame 
        temp,img = find_centroid(net, frame, scale, (416,416))
        # Calculate Euclid distance from new centroid to old centroid
        for  centroid in temp:
            # centroid = np.array(centroid)
            old_centroid = np.array(all_centroid)
            dists = np.sqrt(np.sum(centroid**2) - 2*old_centroid.dot(centroid.T) + np.sum(old_centroid**2,axis=1))
            dists = list(dists)
            # check if dists smaller than threshold
            chosen_dists = [i for i in dists if i <= thresh_dist]
            # if it is a new ID
            if not chosen_dists:
                c_dict[ID_count] = centroid
                ID_count+=1
            else:
                idx = dists.index(min(chosen_dists))
                pre_ID = [key for key,value in c_dict.items() if list(value) == list(all_centroid[idx])][0] 
                c_dict[pre_ID] = centroid
        for key,value in c_dict.items():
            cv2.putText(img,str(key),tuple(value), cv2.FONT_HERSHEY_SIMPLEX , 1,(255,255,255),5)
        all_centroid = temp[:]
        cv2.namedWindow('image',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('image', 800,600)
        cv2.imshow('image',img)
    count += 1
    if count == thresh_fps*30:
        count = 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [6]:
import numpy as np
dicta = {'a':[1,2],'b':[3,4]}
b = [key for key,value in dicta.items() if value == [1,2] ]
a = np.array([1,2])
b = np.array([2,3])
print(list(a)==list(b))

False


In [24]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
def applyFilter(frame, min, max):

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)

    mask = cv2.inRange(frame, min, max)
#     mask = cv2.bitwise_not(mask)

    filtered_frame = cv2.bitwise_and(frame, frame, mask=mask)

    return cv2.cvtColor(filtered_frame, cv2.COLOR_LAB2BGR)
img = cv2.imread('test.png')
low = np.array([0,128,128])
high =  np.array([10,20,120])
frame = applyFilter(img, low,high)
cv2.imshow('frame',frame)
cv2.waitKey()

-1

In [41]:
import numpy as np
X = {(0, 0): 1.0, (0, 1): 0.0, (0, 2): 0.0, (1, 0): 0.0, (1, 1): 1.0, (1, 2): 0.0, (2, 0): 1.0, (2, 1): 0.0, (2, 2): 0.0}
o = {0: -0.0, 1: -0.0, 2: -0.0, 3: -0.0, 4: -0.0, 5: -0.0, 6: -0.0, 7: -0.0, 8: -0.0, 9: -0.0}
import numpy as np
def dict2arr(dictionary):
    # take all keys in dict
    l = list(dictionary.keys())
    # if result is 1D tensor
    if type(l[0])== int:
        result = np.zeros((len(dictionary)))
    # if result is nD tensor with n > 1
    else:
        # take the maximum shape, then plus 1 to generate correct shape
        shape = [i+1 for i in max(l)]
        result = np.zeros(shape)
    # just let the key is index and value is value of result
    for k,v in dictionary.items():
        result[k]=v
    return result
dict2arr(o)

array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])

In [39]:

l = list(X.keys())
a = (max(l))
print(list(a)+1)

TypeError: can only concatenate tuple (not "int") to tuple